In [1]:
import pandas as pd
import json
import pprint
import ast
import gzip
from itertools import chain
from surprise import Reader, Dataset
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np

# Retreive the data and create DataFrames for the files

In [11]:
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield eval(l)

In [12]:
data_test = parse('australian_user_reviews.json.gz')

In [13]:
data_test

<generator object parse at 0x7fbdc7d050b0>

In [14]:
full_data = parse('steam_reviews.json.gz')

In [15]:
full_data

<generator object parse at 0x7fbdc7cef660>

In [16]:
game_data = parse('steam_games.json.gz')

In [17]:
game_data

<generator object parse at 0x7fbdc7d05350>

In [18]:
item_data = parse('australian_users_items.json.gz')

In [19]:
test_df = pd.DataFrame(data_test)

In [20]:
# df = pd.DataFrame(full_data)

In [21]:
game_df = pd.DataFrame(game_data)

In [22]:
item_df = pd.DataFrame(item_data)

In [23]:
item_df

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [24]:
test_df['reviews'][0]

[{'funny': '',
  'posted': 'Posted November 5, 2011.',
  'last_edited': '',
  'item_id': '1250',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'},
 {'funny': '',
  'posted': 'Posted July 15, 2011.',
  'last_edited': '',
  'item_id': '22200',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': "It's unique and worth a playthrough."},
 {'funny': '',
  'posted': 'Posted April 21, 2011.',
  'last_edited': '',
  'item_id': '43110',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}]

In [25]:
# item_df['items'][1]

In [26]:
game_ids = [ids for ids in item_df['items'][0][0]['item_id']]

In [27]:
game_ids

['1', '0']

In [28]:
# for ids in range(0,len(game_ids)):
#     int(item_df['items'][0][ids]['item_id'])
#     for game_id in int(item_df['items'][0][ids]['item_id']:
#         all_game_id.append(game_id)

# Data Cleaning

This saves all of the game ids a user has into a 2d list - this will be used to save the info for the rec system

In [29]:
all_game_id = []
for ids in item_df['items']:
    temp_list = []
    for count in range(len(ids)):
        #print(ids[count]['item_id'])
        temp_list.append(int(ids[count]['item_id']))
    #print(temp_list)
    all_game_id.append(temp_list)
    #print(len(ids))

In [30]:
len(all_game_id[1])

888

In [31]:
item_df['game_ids'] = all_game_id

In [32]:
item_df.head()

,user_id,items_count,steam_id,user_url,items,game_ids
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 20, 30, 40, 50, 60, 70, 130, 300, 240, 38..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 80, 100, 300, 30, 40, 60, 240, 280, 360, ..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest...","[1200, 1230, 1280, 1520, 220, 320, 340, 360, 3..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 20, 30, 40, 50, 60, 70, 130, 80, 100, 300..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea...","[300, 20, 50, 70, 130, 10, 30, 40, 60, 80, 100..."


Create a new column called user_ids to replace the old user id

In [140]:
item_df['user_ids'] = item_df.index

In [141]:
item_df.head()

,user_id,items_count,steam_id,user_url,items,game_ids,user_ids
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 20, 30, 40, 50, 60, 70, 130, 300, 240, 38...",0
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 80, 100, 300, 30, 40, 60, 240, 280, 360, ...",1
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest...","[1200, 1230, 1280, 1520, 220, 320, 340, 360, 3...",2
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 20, 30, 40, 50, 60, 70, 130, 80, 100, 300...",3
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea...","[300, 20, 50, 70, 130, 10, 30, 40, 60, 80, 100...",4


This makes it so the game id explodes and saves the user id to the game they have played

In [46]:
game_df = game_df.rename(columns={'id':'game_ids'})

In [48]:
clean_df = item_df[['user_ids','game_ids']]

In [49]:
clean_df = clean_df.explode('game_ids')

In [50]:
clean_df.reset_index(inplace=True,drop=True)

In [51]:
clean_df

,user_ids,game_ids
0,0,10
1,0,20
2,0,30
3,0,40
4,0,50
...,...,...
5170010,88308,373330
5170011,88308,388490
5170012,88308,521570
5170013,88308,519140


Creates a new df of game_df to allow for merging the DataFrames together

In [53]:
new_game_df = game_df[['app_name','game_ids','title']]

In [54]:
new_game_df = new_game_df[new_game_df['game_ids'].notna()]

Change the game_ids to int so merging can match the values properly

In [55]:
new_game_df=new_game_df.astype({'game_ids':'int'})

Merge the DataFrames

In [57]:
merged_df = clean_df.merge(new_game_df,how='inner',on=['game_ids'])

In [58]:
clean_df=merged_df[['user_ids','game_ids']]

In [59]:
clean_df['game_ids'].isna().sum()

0

In [60]:
clean_df['owned'] = 1

<ipython-input-60-58837cf90a8d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['owned'] = 1


In [61]:
clean_df

,user_ids,game_ids,owned
0,0,10,1
1,1,10,1
2,3,10,1
3,4,10,1
4,10,10,1
...,...,...,...
4294252,74089,375450,1
4294253,85030,353390,1
4294254,86026,354280,1
4294255,86588,433920,1


# Modeling

### Surprise SVD Matrix Factorization

In [69]:
reader = Reader()
data = Dataset.load_from_df(clean_df,reader)

In [70]:
dataset = data.build_full_trainset()
print('Number of users: ', dataset.n_users, '\n')
print('Number of items: ', dataset.n_items)

Number of users:  69304 

Number of items:  9192


In [71]:
params = {'n_factors': [20, 50, 100],
         'reg_all': [0.02, 0.05, 0.1]}
g_s_svd = GridSearchCV(SVD,param_grid=params,n_jobs=-1)
g_s_svd.fit(data)

In [99]:
print(g_s_svd.best_score)
print(g_s_svd.best_params)

{'rmse': 0.0012502239310262877, 'mae': 0.00018972092263369098}
{'rmse': {'n_factors': 20, 'reg_all': 0.1}, 'mae': {'n_factors': 20, 'reg_all': 0.1}}


### KNN Model - currently not able to run without crashing

In [73]:
# knn_basic = KNNBasic(sim_options={'name':'pearson', 'user_based':True})
# cv_knn_basic = cross_validate(knn_basic, data, n_jobs=-1)

In [74]:
# for i in cv_knn_basic.items():
#     print(i)
# print('-----------------------')
# print(np.mean(cv_knn_basic['test_rmse']))

In [77]:
svd = SVD(n_factors= 20, reg_all=0.1)
svd.fit(dataset)

In [78]:
svd.predict(2, 4)

Prediction(uid=2, iid=4, r_ui=None, est=1.0000453870448958, details={'was_impossible': False})

In [123]:
merged_df['game_ids'].value_counts().head(50)

730       43776
4000      43301
304930    38682
550       37044
105600    29239
230410    25807
620       24465
240       24220
218620    23729
301520    21785
49520     21029
218230    20726
340       19826
227940    19721
400       18774
220       18307
236390    18172
333930    17233
208090    16489
8190      16272
252490    16201
224260    15979
219640    15930
320       15854
322330    15571
8930      15303
24240     15202
33930     15084
55230     14753
1250      14216
35420     14000
304050    13969
360       13327
380       13275
22380     13131
420       13060
42910     12239
200210    12105
8870      11967
252950    11605
204360    11511
113200    11400
222880    11368
221100    10923
17410     10846
291480    10794
265630    10723
407530    10493
323370    10484
291550    10322
Name: game_ids, dtype: int64

Create a new user using 4 random games from the list above of the top 50 most popular games

In [125]:
user_play = [{'user_ids': 100000, 'game_ids': 730, 'owned': 1},
 {'user_ids': 100000, 'game_ids': 400, 'owned': 1},
 {'user_ids': 100000, 'game_ids': 204360, 'owned': 1},
 {'user_ids': 100000, 'game_ids': 17410, 'owned': 1}]

Add the new user to the dataset

In [127]:
new_games_df = clean_df.append(user_play,ignore_index=True)
new_data = Dataset.load_from_df(new_games_df,reader)

In [128]:
svd_ = SVD(n_factors= 20, reg_all=0.1)
svd_.fit(new_data.build_full_trainset())

Loop through all of the games and make predictions on those games for the new user

In [130]:
list_of_games = []
for g_id in clean_df['game_ids'].unique():
    list_of_games.append((g_id,svd_.predict(100000,g_id)[3]))

In [131]:
game_pred = sorted(list_of_games, key=lambda x:x[1], reverse=True)

A function that predicts new recommendations for the user

In [133]:
def recommended_games(user_play,game_title_df,n):
        for idx, rec in enumerate(user_play):
            title = game_title_df.loc[game_title_df['game_ids'] == int(rec[0])]['app_name']
            first_title = title.iloc[0]
            print('Recommendation # ', idx+1, ': ', first_title, '\n')
            n-= 1
            if n == 0:
                break

In [138]:
recommended_games(game_pred,merged_df,5)

Recommendation #  1 :  The ORPHEUS Ruse 

Recommendation #  2 :  Mahjong Roadshow™ 

Recommendation #  3 :  Andromedum 

Recommendation #  4 :  Driver Fusion Premium 

Recommendation #  5 :  Awareness Rooms 

